In [4]:
import sqlite3
import pandas as pd

In [5]:
conn = sqlite3.connect('lahman2016.sqlite')

In [7]:
query = '''select * from Teams
inner join TeamsFranchises
on Teams.franchID == TeamsFranchises.franchID
where Teams.G >= 150 and TeamsFranchises.active == 'Y';
'''

In [ ]:
Teams = conn.execute(query).fetchall()

In [ ]:
teams_df = pd.DataFrame(Teams)
teams_df.head(5)

In [ ]:
cols = ['yearID','lgID','teamID','franchID','divID',
        'Rank','G','Ghome','W','L','DivWin','WCWin',
        'LgWin','WSWin','R','AB','H','2B','3B','HR',
        'BB','SO','SB','CS','HBP','SF','RA','ER','ERA',
        'CG','SHO','SV','IPouts','HA','HRA','BBA','SOA',
        'E','DP','FP','name','park','attendance','BPF',
        'PPF','teamIDBR','teamIDlahman45','teamIDretro',
        'franchID','franchName','active','NAassoc']
teams_df.columns = cols

In [ ]:
print(teams_df.head(2))

In [ ]:
len(teams_df)

In [ ]:
drop_cols = ['lgID','franchID','divID','Rank','Ghome',
             'L','DivWin','WCWin','LgWin','WSWin','SF',
             'name','park','attendance','BPF','PPF',
             'teamIDBR','teamIDlahman45','teamIDretro',
             'franchID','franchName','active','NAassoc']
df = teams_df.drop(drop_cols,axis=1)

In [ ]:
df.head(2)

In [ ]:
print(df.isnull().sum(axis=0).tolist())
#axis=0: act on all rows for each column
#axis=1: act on all columns for each row

In [ ]:
df = df.drop(['CS','HBP'],axis=1)

In [ ]:
df['SO'] = df['SO'].fillna(df['SO'].median())
df['DP'] = df['DP'].fillna(df['DP'].median())

In [ ]:
print(df.isnull().sum(axis=0).tolist())

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.hist(df['W'])
plt.xlabel('wins')
plt.title('histogram of wins')

In [ ]:
df['W'].mean()

In [9]:
#create bins for each win values
def assign_win_bins(w):
    if w < 50:
        return 1
    if w >= 50 and w <= 69:
        return 2
    if w >= 70 and w <= 89:
        return 3
    if w >= 90 and w <= 109:
        return 4
    if w >= 110:
        return 5

In [ ]:
#apply to the 'W' column
df['win_bins'] = df['W'].apply(assign_win_bins)

In [ ]:
plt.scatter(df['yearID'],df['W'],c=df['win_bins'])
plt.title('wins per year')
plt.xlabel('year')
plt.ylabel('wins')

In [ ]:
df = df[df['yearID']>1900]

In [ ]:
runs_per_year = {}
games_per_year = {}

for i, row in df.iterrows():
    year = row['yearID']
    runs = row['R']
    games = row['G']
    if year in runs_per_year:
        runs_per_year[year] = runs_per_year[year] + runs
        games_per_year[year] = games_per_year[year] + games
    else:
        runs_per_year[year] = runs
        games_per_year[year] = games

In [ ]:
print(runs_per_year)

{1961: 12942, 1962: 14461, 1963: 12780, 1964: 13124, 1965: 12946, 1966: 12900, 1967: 12210, 1968: 11109, 1969: 15850, 1970: 16880, 1971: 15073, 1972: 13706, 1973: 16376, 1974: 16046, 1975: 16295, 1976: 15492, 1977: 18803, 1978: 17251, 1979: 18713, 1980: 18053, 1982: 18110, 1983: 18170, 1984: 17921, 1985: 18216, 1986: 18545, 1987: 19883, 1988: 17380, 1989: 17405, 1990: 17919, 1991: 18127, 1992: 17341, 1993: 20864, 1996: 22831, 1997: 21604, 1998: 23297, 1999: 24691, 2000: 24971, 2001: 23199, 2002: 22408, 2003: 22978, 2004: 23376, 2005: 22325, 2006: 23599, 2007: 23322, 2008: 22585, 2009: 22419, 2010: 21308, 2011: 20808, 2012: 21017, 2013: 20255, 2014: 19761, 2015: 20647, 2016: 21744, 1904: 9307, 1905: 9640, 1906: 8322, 1907: 8186, 1908: 8423, 1909: 8805, 1910: 9580, 1911: 11164, 1912: 11154, 1913: 9964, 1914: 9408, 1915: 9444, 1916: 8889, 1917: 8948, 1920: 10762, 1921: 11935, 1922: 12059, 1923: 11874, 1924: 11722, 1925: 12596, 1926: 11443, 1927: 11745, 1928: 11645, 1929: 12747, 1930: 13695, 1931: 11892, 1932: 12116, 1933: 10288, 1934: 11325, 1935: 10498, 1936: 12846, 1937: 12070, 1938: 11260, 1939: 11876, 1940: 11568, 1941: 11168, 1942: 9457, 1943: 9694, 1944: 10353, 1945: 9729, 1946: 9953, 1947: 10827, 1948: 11328, 1949: 11426, 1950: 12013, 1951: 11268, 1952: 10349, 1953: 11426, 1954: 10827, 1955: 11069, 1956: 11031, 1957: 10636, 1958: 10578, 1959: 10853, 1960: 10664}


In [ ]:
print(games_per_year)

{1961: 2860, 1962: 3242, 1963: 3238, 1964: 3252, 1965: 3246, 1966: 3230, 1967: 3240, 1968: 3250, 1969: 3892, 1970: 3888, 1971: 3876, 1972: 3718, 1973: 3886, 1974: 3890, 1975: 3868, 1976: 3878, 1977: 4206, 1978: 4204, 1979: 4198, 1980: 4210, 1982: 4214, 1983: 4218, 1984: 4210, 1985: 4206, 1986: 4206, 1987: 4210, 1988: 4200, 1989: 4212, 1990: 4210, 1991: 4208, 1992: 4212, 1993: 4538, 1996: 4534, 1997: 4532, 1998: 4864, 1999: 4856, 2000: 4858, 2001: 4858, 2002: 4852, 2003: 4860, 2004: 4856, 2005: 4862, 2006: 4858, 2007: 4862, 2008: 4856, 2009: 4860, 2010: 4860, 2011: 4858, 2012: 4860, 2013: 4862, 2014: 4860, 2015: 4858, 2016: 4856, 1904: 2498, 1905: 2474, 1906: 2307, 1907: 2317, 1908: 2488, 1909: 2482, 1910: 2498, 1911: 2474, 1912: 2464, 1913: 2468, 1914: 2512, 1915: 2490, 1916: 2494, 1917: 2494, 1920: 2468, 1921: 2458, 1922: 2476, 1923: 2466, 1924: 2462, 1925: 2456, 1926: 2468, 1927: 2472, 1928: 2462, 1929: 2458, 1930: 2468, 1931: 2472, 1932: 2466, 1933: 2303, 1934: 2297, 1935: 2158, 1936: 2476, 1937: 2478, 1938: 2297, 1939: 2462, 1940: 2472, 1941: 2488, 1942: 2300, 1943: 2476, 1944: 2484, 1945: 2313, 1946: 2484, 1947: 2486, 1948: 2474, 1949: 2480, 1950: 2476, 1951: 2478, 1952: 2478, 1953: 2480, 1954: 2474, 1955: 2468, 1956: 2478, 1957: 2470, 1958: 2470, 1959: 2476, 1960: 2472}


In [ ]:
mlb_runs_per_game = {}

for k,v in games_per_year.items():
    year = k
    games = v
    runs = runs_per_year[year]
    mlb_runs_per_game[year] = runs/games

In [ ]:
print(mlb_runs_per_game)

{1961: 4.525174825174825, 1962: 4.460518198642813, 1963: 3.9468807906114884, 1964: 4.035670356703567, 1965: 3.9882932840418976, 1966: 3.9938080495356036, 1967: 3.7685185185185186, 1968: 3.4181538461538463, 1969: 4.07245632065776, 1970: 4.341563786008231, 1971: 3.8888028895768834, 1972: 3.6863905325443787, 1973: 4.214101904271745, 1974: 4.124935732647815, 1975: 4.21277145811789, 1976: 3.99484270242393, 1977: 4.470518307180218, 1978: 4.103472882968601, 1979: 4.457598856598381, 1980: 4.2881235154394295, 1982: 4.297579496915045, 1983: 4.307728781412992, 1984: 4.256769596199525, 1985: 4.330955777460771, 1986: 4.409177365668093, 1987: 4.722802850356294, 1988: 4.1380952380952385, 1989: 4.132241215574549, 1990: 4.256294536817102, 1991: 4.307747148288973, 1992: 4.1170465337132, 1993: 4.597620096959012, 1996: 5.0355094838994265, 1997: 4.766990291262136, 1998: 4.789679276315789, 1999: 5.0846375617792425, 2000: 5.140181144503911, 2001: 4.7754219843557015, 2002: 4.6183017312448476, 2003: 4.72798353909465, 2004: 4.813838550247117, 2005: 4.591731797614151, 2006: 4.857760395224372, 2007: 4.7967914438502675, 2008: 4.650947281713345, 2009: 4.612962962962963, 2010: 4.3843621399176955, 2011: 4.283244133388226, 2012: 4.324485596707819, 2013: 4.1659810777457835, 2014: 4.0660493827160495, 2015: 4.250102923013586, 2016: 4.477759472817134, 1904: 3.7257806244995995, 1905: 3.896523848019402, 1906: 3.6072821846553964, 1907: 3.5330168321104876, 1908: 3.385450160771704, 1909: 3.5475423045930703, 1910: 3.8350680544435547, 1911: 4.512530315278901, 1912: 4.526785714285714, 1913: 4.037277147487845, 1914: 3.745222929936306, 1915: 3.792771084337349, 1916: 3.5641539695268647, 1917: 3.587810745789896, 1920: 4.360615883306321, 1921: 4.855573637103336, 1922: 4.870355411954765, 1923: 4.815085158150851, 1924: 4.761169780666125, 1925: 5.128664495114006, 1926: 4.636547811993517, 1927: 4.75121359223301, 1928: 4.729894394800975, 1929: 5.185923515052888, 1930: 5.5490275526742305, 1931: 4.810679611650485, 1932: 4.913219789132198, 1933: 4.467216673903604, 1934: 4.9303439268611235, 1935: 4.86468952734013, 1936: 5.188206785137318, 1937: 4.870863599677159, 1938: 4.902046147148455, 1939: 4.823720552396425, 1940: 4.679611650485437, 1941: 4.488745980707395, 1942: 4.111739130434783, 1943: 3.9151857835218093, 1944: 4.167874396135265, 1945: 4.206225680933852, 1946: 4.006843800322061, 1947: 4.3551890587288815, 1948: 4.5788197251414715, 1949: 4.607258064516129, 1950: 4.851777059773829, 1951: 4.547215496368039, 1952: 4.17635189669088, 1953: 4.607258064516129, 1954: 4.376313662085691, 1955: 4.485008103727715, 1956: 4.451573849878935, 1957: 4.306072874493927, 1958: 4.282591093117409, 1959: 4.383279483037157, 1960: 4.313915857605178}


In [ ]:
#create lists from mlb_runs_per_game dictionary
lists = sorted(mlb_runs_per_game.items())
x,y = zip(*lists)
#x is the list of keys, y is the list of values

In [ ]:
#create line plot of mlb_runs_per_game vs year
plt.plot(x,y)
plt.title('mlb yearly runs per game')
plt.xlabel('year')
plt.ylabel('mlb runs per game')

In [13]:
def assign_label(year):
    if year < 1920:
        return 1
    elif year >= 1920 and year <= 1941:
        return 2
    elif year >= 1942 and year <= 1945:
        return 3
    elif year >= 1946 and year <= 1962:
        return 4
    elif year >= 1963 and year <= 1976:
        return 5
    elif year >= 1977 and year <= 1992:
        return 6
    elif year >= 1993 and year <= 2009:
        return 7
    elif year >= 2010:
        return 8

In [ ]:
df['year_label'] = df['yearID'].apply(assign_label)

In [ ]:
dummy_df = pd.get_dummies(df['year_label'],prefix='era')

In [ ]:
dummy_df.head()

In [ ]:
df = pd.concat([df,dummy_df],axis=1)

In [ ]:
df.head(2)

In [ ]:
def assign_mlb_rpg(year):
    return mlb_runs_per_game[year]
df['mlb_rpg'] = df['yearID'].apply(assign_mlb_rpg)

In [16]:
def assign_decade(year):
    if year < 1920:
        return 1910
    elif year >= 1920 and year <= 1929:
        return 1920
    elif year >= 1930 and year <= 1939:
        return 1930
    elif year >= 1940 and year <= 1949:
        return 1940
    elif year >= 1950 and year <= 1959:
        return 1950
    elif year >= 1960 and year <= 1969:
        return 1960
    elif year >= 1970 and year <= 1979:
        return 1970
    elif year >= 1980 and year <= 1989:
        return 1980
    elif year >= 1990 and year <= 1999:
        return 1990
    elif year >= 2000 and year <= 2009:
        return 2000
    elif year >= 2010:
        return 2010

In [ ]:
df['decade_label'] = df['yearID'].apply(assign_decade)

In [ ]:
decade_df = pd.get_dummies(df['decade_label'],prefix='decade')

In [ ]:
df = pd.concat([df,decade_df],axis=1)

In [ ]:
df = df.drop(['yearID','year_label','decade_label'],axis=1)

In [ ]:
df['R_per_game'] = df['R']/df['G']
df['RA_per_game'] = df['RA']/df['G']

In [ ]:
fig = plt.figure(figsize=(12,6))

ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

ax1.scatter(df['R_per_game'],df['W'],c='blue')
ax1.set_title('runs per game vs wins')
ax1.set_ylabel('wins')
ax1.set_xlabel('runs per game')

ax2.scatter(df['RA_per_game'],df['W'],c='red')
ax2.set_title('runs allowed per game vs wins')
ax2.set_xlabel('runs allowed per game')

In [ ]:
df.corr()['W']

G              0.178958
W              1.000000
R              0.551504
AB             0.233996
H              0.380281
2B             0.262323
3B             0.016970
HR             0.312713
BB             0.345348
SO             0.072069
SB             0.133300
RA            -0.518313
ER            -0.372580
ERA           -0.425833
CG            -0.041788
SHO            0.426892
SV             0.382174
IPouts         0.326087
HA            -0.384893
HRA            0.009880
BBA           -0.284782
SOA            0.259205
E             -0.304725
DP             0.015192
FP             0.319597
win_bins       0.912545
era_1         -0.077104
era_2         -0.083089
era_3         -0.033073
era_4         -0.052278
era_5          0.042122
era_6          0.058439
era_7          0.066049
era_8          0.043453
mlb_rpg        0.006909
decade_1910   -0.077104
decade_1920   -0.055176
decade_1930   -0.052358
decade_1940   -0.050249
decade_1950   -0.047499
decade_1960    0.028594
decade_1970    0.033679
decade_1980    0.043516
decade_1990    0.044150
decade_2000    0.052283
decade_2010    0.043453
R_per_game     0.528489
RA_per_game   -0.550696
Name: W, dtype: float64

In [ ]:
attributes = ['G','R','AB','H','2B','3B','HR','BB','SO','SB','RA','ER','ERA','CG',
'SHO','SV','IPouts','HA','HRA','BBA','SOA','E','DP','FP','era_1','era_2','era_3','era_4','era_5','era_6','era_7','era_8','de

In [ ]:
df.head()


In [19]:
from sklearn.cluster import KMeans
from sklearn import metrics

In [ ]:
score_dict = {}
for i in range(2,11):
    km = KMeans(n_clusters = i, random_state=1)
    l = km.fit_predict(data_attributes)
    score = metrics.silhouette_score(data_attributes,l)
    score_dict[i] = score

In [ ]:
score_dict

In [ ]:
kmeans_model = KMeans(n_clusters = 6, random_state=1)

In [ ]:
#determine euclidian distances for each data points
distances = kmeans_model.fit_transform(data_attributes)

In [ ]:
distances

array([[ 403.64157318,  905.96921371,  418.08948195,  691.28217379,
         881.91035476,  364.27867505],
       [ 176.63291442,  724.86201359,  462.79421258,  528.17076064,
         742.53312859,  356.92059367],
       [ 175.28064133,  760.02048493,  461.22878542,  533.34938643,
         670.03986589,  459.28750389],
       ...,
       [ 645.94940211, 1281.77528634,  315.40745596, 1063.25746178,
        1095.60371693,  628.81413485],
       [ 692.85283233, 1299.34088523,  309.93820869, 1072.79831862,
        1111.7177554 ,  628.41580125],
       [ 738.16587506, 1365.13388024,  349.40577985, 1136.23398531,
        1200.02051407,  665.81041136]])

In [ ]:
#create scatter plot colored by kmeans clusters
labels = kmeans_model.labels_
plt.scatter(distances[:,0],distances[:,1],c=labels)
plt.title('kmeans clusters')

In [ ]:
df['labels'] = labels
attributes.append('labels')
df.head()

In [25]:
numeric_cols = ['G','R','AB','H','2B','3B','HR','BB','SO','SB','RA','ER','ERA','CG','SHO','SV','IPouts','HA','HRA','BBA','SOA','E','DP','FP','era_1','era_2','era_3','era_4','era_5','era_6','era_7','era_8','decade_1910','decade_1920','decade_1930','decade_1940','decade_1950','decade_1960','decade_1970','decade_1980','decade_1990','decade_2000','decade_2010','R_per_game','RA_per_game','mlb_rpg','labels','W']


In [ ]:
data = df[numeric_cols]

In [ ]:
data.head()

0	162	744	5424	1331	218	22	189	681	1068.0	37	...	0	0	0	0	0	4.592593	4.839506	4.525175	5	70
1	162	718	5499	1377	232	35	137	602	917.0	46	...	0	0	0	0	0	4.432099	4.358025	4.460518	0	86
2	161	597	5506	1378	208	38	95	448	916.0	43	...	0	0	0	0	0	3.708075	4.099379	3.946881	0	70
3	162	544	5362	1297	186	27	102	472	920.0	49	...	0	0	0	0	0	3.358025	3.401235	4.035670	0	82
4	162	527	5354	1279	200	36	92	443	973.0	107	...	0	0	0	0	0	3.253086	3.512346	3.988293	0	75
5 rows × 48 columns